## SOLN - noise & mcmc

noise is part of working with data so let's learn how it works

we will cover
- what is noise
- nature of noise in data
- simulate noisy data
- what is mcmc
- mcmc on simulated data

### what is noise

noise in science is a very **__specific__** term (aka johnson nyquist noise) - you can't just say anything is noise. noise comes from the random motion of electrons and is directly proportional to temperature. noise originates in the electronics that are taking data - it does not exist in the source.

### noise in data

noise adds power to the data and is **gaussian**

### simulation - noise

In [ ]:
# simulate a gaussian source

# add noise to the data


### what is mcmc

mcmc stands for markov chain monte carlo. please look into what markov chains are and what monte carlo means. this is just a way for you to see how this works. mcmc is a machine learning algorithm that allows you to fit paramters to a distribution. in other words, you can give it a function and some data and it will estimate the arguments into the function that would produce that data. it chooses the parameters that minimize the least-squares difference between the distribution and the data (aka minimizing the loss).

how does it do this? imagine a hilly terrain where walking in one dimension is changing one parameter (say, x) and the other direction is another parameter (say, y). we want to figure out which combination of x and y represents our data. we want to find f(x, y) that produced the data. this hilly terrain is known as the parameter space and the correct combination of x and y is on top of the highest hill. what does the f(x, y) represent? it is the loss function but negated so that the minimum loss is at the top of hte highest hill.

we want to start somewhere random and make tiny random changes to x and y to see which changes are effective in decreasing the loss with respect to where we are currently standing. this keeps going until we reach the top of the highest hill. we know we are there because any change we make would cause the f(x, y) of the new spot to be less than the one we currenly stand on.

the implementation we will use does something clever here. it will randomly take that step downhill. why? in case we are stuck on a local maxima - a hill that isn't the tallest hill. it also has a No U-Turns (NUTS) sampler that does not retrace steps to save time and energy.

this was a LOT of ML jargon but it's helpful to know how your algorithms work. this is a very surface level view of mcmc so please read more online if you're interested, it's super cool.

### simulation - mcmc

In [ ]:
# mcmc implementation